In [179]:
# Dependencies and Setup
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import time # use for annotating the date of the API pull in graphics using '.now'

import openbrewerydb # a wrapper for accessing the Open Brewery DB API

import gmaps
import gmaps.datasets

import gmaps.geojson_geometries

from matplotlib.cm import viridis
from matplotlib.colors import to_hex

#config
from config import api_key

from opencage.geocoder import OpenCageGeocode
from pprint import pprint


gmaps.configure(api_key=api_key)
# print(api_key) # comment out before saving or uploading to github

In [181]:
brewery_data_df = openbrewerydb.load(state=None)
brewery_data_df.count() # there are approximately 8,000 breweries in the US as of 2019

brewery_type    8029
city            7924
country         8029
id              8029
latitude        4737
longitude       4737
name            8029
phone           8029
postal_code     7924
state           7924
street          8029
tag_list        8029
updated_at      8029
website_url     8029
dtype: int64

#### Note: The count of breweries with latitude and longitude is 4,737 - this will be the (inital) "demo" list of breweries  for mapping
* The brewery_data_df has over 8,000 entries; however, only 4,737 have a lat/lon entry
* A better location choice (one that gets the most brewery locations) is city, state and/or postal_code
* Follow-up work will create missing latitude and longitude from the available city, state, postal_code information for those having an "NaN" which should increase the number of breweries back up to 7,924.
* Is is assumed that the relative density of breweries as would not be depicted on a map would be signficantly different for cities with a larger number of breweries (~ 23 or more).

In [182]:
# View brewery_data_df looking for NaN for latitude and longitude 
brewery_data_df.head(120) # note the brewpub in Prescott Valley, AZ (entry 92) has NaN for latitude and longitude 

,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,street,tag_list,updated_at,website_url
0,micro,Birmingham,United States,2,33.524521,-86.774322,Avondale Brewing Co,2057775456,35222-1932,Alabama,201 41st St S,[],2018-08-23T23:19:57.825Z,http://www.avondalebrewing.com
1,micro,Tuscaloosa,United States,4,33.198491,-87.562155,Band of Brothers Brewing Company,2052665137,35401-4653,Alabama,1605 23rd Ave,[],2018-08-23T23:19:59.462Z,http://www.bandofbrosbrewing.com
2,micro,Birmingham,United States,44,33.512849,-86.791400,Trim Tab Brewing,2057030536,35233-3401,Alabama,2721 5th Ave S,[],2018-08-23T23:20:31.423Z,http://www.trimtabbrewing.com
3,micro,Huntsville,United States,46,34.727752,-86.593201,Yellowhammer Brewery,2569755950,35805-3046,Alabama,2600 Clinton Ave W,[],2018-08-23T23:20:33.102Z,http://www.yellowhammerbrewery.com
4,micro,Wasilla,United States,55,61.575269,-149.412710,Bearpaw River Brewing Co,,99654-7679,Alaska,4605 E Palmer Wasilla Hwy,[],2018-08-23T23:20:40.743Z,http://bearpawriverbrewing.com
5,micro,Anchorage,United States,76,61.138489,-149.879076,King Street Brewing Co,9073365464,99515,Alaska,9050 King Street,[],2018-08-23T23:20:57.179Z,http://www.kingstreetbrewing.com
6,micro,Tucson,United States,94,32.246737,-110.992751,1912 Brewing,5202564851,85745-1444,Arizona,2045 N Forbes Blvd Ste 105,[],2018-08-23T23:21:11.302Z,http://www.1912brewing.com
7,contract,Scottsdale,United States,98,33.497262,-111.924474,Bad Water Brewing,5207459175,85251-3914,Arizona,4216 N Brown Ave,[],2018-08-23T23:21:15.169Z,http://www.badwaterbrewing.com
8,brewpub,Chandler,United States,104,33.305346,-111.911126,BJs Restaurant & Brewery - Chandler,4809170631,85226-5175,Arizona,3155 W Chandler Blvd,[],2018-08-23T23:21:21.165Z,http://www.bjsrestaurants.com
9,micro,Tucson,United States,107,32.201608,-110.821779,BlackRock Brewers,5202073203,85710-6767,Arizona,1664 S Research Loop Ste 200,[],2018-08-23T23:21:23.794Z,http://www.brb.beer


In [183]:
# Create a subset of the brewery_data_df called brewery_locacation_df that drops those rows that have NaN entries for 'latitude' and 'longitude' - this should reduce the inital list to 4,737
# Follow-up work will go back and get the LAT and LON for those missing locations after getting this inital list of 4,737 to plot in gmaps
brewery_location_df = brewery_data_df.dropna(subset=['latitude', 'longitude'])
brewery_location_df


,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,street,tag_list,updated_at,website_url
0,micro,Birmingham,United States,2,33.524521,-86.774322,Avondale Brewing Co,2057775456,35222-1932,Alabama,201 41st St S,[],2018-08-23T23:19:57.825Z,http://www.avondalebrewing.com
1,micro,Tuscaloosa,United States,4,33.198491,-87.562155,Band of Brothers Brewing Company,2052665137,35401-4653,Alabama,1605 23rd Ave,[],2018-08-23T23:19:59.462Z,http://www.bandofbrosbrewing.com
2,micro,Birmingham,United States,44,33.512849,-86.791400,Trim Tab Brewing,2057030536,35233-3401,Alabama,2721 5th Ave S,[],2018-08-23T23:20:31.423Z,http://www.trimtabbrewing.com
3,micro,Huntsville,United States,46,34.727752,-86.593201,Yellowhammer Brewery,2569755950,35805-3046,Alabama,2600 Clinton Ave W,[],2018-08-23T23:20:33.102Z,http://www.yellowhammerbrewery.com
4,micro,Wasilla,United States,55,61.575269,-149.412710,Bearpaw River Brewing Co,,99654-7679,Alaska,4605 E Palmer Wasilla Hwy,[],2018-08-23T23:20:40.743Z,http://bearpawriverbrewing.com
5,micro,Anchorage,United States,76,61.138489,-149.879076,King Street Brewing Co,9073365464,99515,Alaska,9050 King Street,[],2018-08-23T23:20:57.179Z,http://www.kingstreetbrewing.com
6,micro,Tucson,United States,94,32.246737,-110.992751,1912 Brewing,5202564851,85745-1444,Arizona,2045 N Forbes Blvd Ste 105,[],2018-08-23T23:21:11.302Z,http://www.1912brewing.com
7,contract,Scottsdale,United States,98,33.497262,-111.924474,Bad Water Brewing,5207459175,85251-3914,Arizona,4216 N Brown Ave,[],2018-08-23T23:21:15.169Z,http://www.badwaterbrewing.com
8,brewpub,Chandler,United States,104,33.305346,-111.911126,BJs Restaurant & Brewery - Chandler,4809170631,85226-5175,Arizona,3155 W Chandler Blvd,[],2018-08-23T23:21:21.165Z,http://www.bjsrestaurants.com
9,micro,Tucson,United States,107,32.201608,-110.821779,BlackRock Brewers,5202073203,85710-6767,Arizona,1664 S Research Loop Ste 200,[],2018-08-23T23:21:23.794Z,http://www.brb.beer


In [184]:
# Use the existing brewery latitude and longitude to create a heatmap using gmaps
# note gmaps has three possible views, the default is a "TERRAN" but "SATELLITE" or  "HYBRID"
locations = brewery_location_df[['latitude', 'longitude']]

fig = gmaps.figure(map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [185]:
# list the brewery counts by city using count of postal_codes
breweries_by_state_city_df = brewery_data_df.groupby(['state', 'city'])[['postal_code']].count()
# Sort by state and then city in decending order
breweries_by_state_city_df.sort_values(by='postal_code',ascending=False)

,,postal_code
state,city,
Colorado,Denver,93
California,San Diego,91
Oregon,Portland,86
Washington,Seattle,73
Illinois,Chicago,65
Texas,Austin,54
California,San Francisco,39
Minnesota,Minneapolis,38
Ohio,Cincinnati,35


#### Note: There are 3,494 unique cities in the USA with breweries.  
* The top 30 cities (having 23 or more breweries) are listed in the top half (above the ..) and range from Denver, Colorado with 93 breweries to Richmond, Nashville and Spokane all with 23. 
* The bottom 30 cities each have only one brewery - note there are many more cities that have only one brewery
Question: Using Demographic data from US Census, are there any significant differences between cities in the top 30 vs. cities in the bottom 30?

In [124]:
#--------------------------------------------------------------------
# WORK IN PROGRESS
#--------------------------------------------------------------------

# Add GeoJSON to a map to draw a thematic map which displays shading in proportion to the measurement
# being displayed on the map, such as population density or per-capita income.

countries_geojson = gmaps.geojson_geometries.load_geometry('us-states')
fig = gmaps.figure()
geojson_layer = gmaps.geojson_layer(countries_geojson)
fig.add_layer(geojson_layer)
fig

# Scale the number of breweries per state to lie between 0 and 1
min_brewery = min(state2brewery_count.values())
max_brewery = max(state2brewery_count.values())
brewery_range = max_brewery - min_brewery
def calculate_color(brew_index):
    """ 
    Convert the brew_index coefficient to a color
    """
# make brew_index a number between 0 and 1
normalized_brew_index = (brewery_count - min_brewery) / brewery_range
# invert brewery_index so that high inequality gives dark color
inverse_brewery = 1.0 - normalized_brewery_index
# transform the brewery_index coefficient to a matplotlib color
mpl_color = viridis(inverse_gini)
# transform from a matplotlib color to a valid CSS color
gmaps_color = to_hex(mpl_color, keep_alpha=False)
return gmaps_color

Figure(layout=FigureLayout(height='420px'))

## References:

### Literature Review of Craft Brewery Statistics
* https://www.brewersassociation.org/insights/shifting-demographics-among-craft-drinkers/
* https://www.brewersassociation.org/category/insights/
* https://www.brewersassociation.org/statistics/by-state/
* https://www.brewersassociation.org/statistics/national-beer-sales-production-data/
* https://www.brewersassociation.org/statistics/economic-impact-data/
* https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2018/08/Brewers-Association-Economic-Impact-Study-Methodology.pdf
* https://www.brewersassociation.org/statistics/number-of-breweries/
* https://s3-us-west-2.amazonaws.com/brewersassoc/wp-content/uploads/2018/08/State-by-State-Breakdown-2017.pdf
* https://www.brewersassociation.org/statistics/market-segments/
* https://www.brewersassociation.org/government-affairs/laws/barrel-cap-laws/


### Brewery API sites
* https://www.openbrewerydb.org/ # includes a wrapper for making API calls by state, type brewery etc.

### Other API sites considered / used
* https://www.zipcodeapi.com/ #site that translates address, city, state into lat lon coordinates
* https://opencagedata.com #site that translates address, city, state into lat lon coordinates

### Mapping using Google Maps
* https://buildmedia.readthedocs.org/media/pdf/jupyter-gmaps/latest/jupyter-gmaps.pdf

